**NEURAL NETWORKS FROM SCRATCH**

Some utility functions

In [1]:
import numpy as np

import os

os.environ['OMP_NUM_THREADS'] = '11'

def tanh(x):
    return np.tanh(x);

def tanh_prime(x):
    return 1-np.tanh(x)**2;

def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2));

def mse_prime(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size;

def relu(x):
    return np.maximum(0, x)

def relu_prime(x):
    return np.where(x > 0, 1, 0)

Abstract class Layer

In [2]:
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    def forward_propagation(self, input):
        raise NotImplementedError

    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

Fully-connected layer

In [3]:
class FCLayer(Layer):
    def __init__(self, input_size, output_size):
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5

    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

Droput Layer

In [4]:
class DropoutLayer(Layer):
    def __init__(self, p):
        self.p = p
        self.mask = None

    def forward_propagation(self, input_data):
        self.mask = np.random.rand(*input_data.shape) > self.p
        self.output = input_data * self.mask / (1 - self.p)
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        return output_error * self.mask / (1 - self.p)

Activation layer

In [5]:
class ActivationLayer(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error
    
    def forward_propagation_dropout(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    def backward_propagation_dropout(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error

Network class

In [6]:
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    def add(self, layer):
        self.layers.append(layer)

    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    def predict(self, input_data):
        samples = len(input_data)
        result = []
        for i in range(samples):
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)
        return result

    def fit(self, x_train, y_train, epochs, learning_rate):
        samples = len(x_train)
        for i in range(epochs):
            err = 0
            for j in range(samples):
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)
                err += self.loss(y_train[j], output)
                error = self.loss_prime(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)
            err /= samples
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))

Solve the XOR problem

In [7]:
x_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
y_train = np.array([[[0]], [[1]], [[1]], [[0]]])
np.random.seed(10)
net = Network()
net.add(FCLayer(2, 10))
net.add(ActivationLayer(relu, relu_prime))  # Use ReLU activation
net.add(FCLayer(10, 1))
net.add(ActivationLayer(relu, relu_prime))
net.add(DropoutLayer(p=0.1))  # Add dropout layer
net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=500, learning_rate=0.1)

epoch 1/500   error=0.439734
epoch 2/500   error=0.337554
epoch 3/500   error=0.503549
epoch 4/500   error=0.302155
epoch 5/500   error=0.274435
epoch 6/500   error=0.266073
epoch 7/500   error=0.261782
epoch 8/500   error=0.256543
epoch 9/500   error=0.502442
epoch 10/500   error=0.253332
epoch 11/500   error=0.251993
epoch 12/500   error=0.250028
epoch 13/500   error=0.251212
epoch 14/500   error=0.250602
epoch 15/500   error=0.250329
epoch 16/500   error=0.250172
epoch 17/500   error=0.500067
epoch 18/500   error=0.250059
epoch 19/500   error=0.250039
epoch 20/500   error=0.250019
epoch 21/500   error=0.250009
epoch 22/500   error=0.250005
epoch 23/500   error=0.250000
epoch 24/500   error=0.250003
epoch 25/500   error=0.250001
epoch 26/500   error=0.250001
epoch 27/500   error=0.250000
epoch 28/500   error=0.250000
epoch 29/500   error=0.250000
epoch 30/500   error=0.250000
epoch 31/500   error=0.250000
epoch 32/500   error=0.250000
epoch 33/500   error=0.250000
epoch 34/500   erro

Solve the MNIST problem

In [ ]:
from keras.datasets import mnist
from keras.utils import to_categorical
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 1, 28*28)
x_train = x_train.astype('float32')
x_train /= 255
y_train = to_categorical(y_train)
x_test = x_test.reshape(x_test.shape[0], 1, 28*28)
x_test = x_test.astype('float32')
x_test /= 255
y_test = to_categorical(y_test)
np.random.seed(10)
net = Network()
net.add(FCLayer(28*28, 100))
net.add(ActivationLayer(relu, relu_prime))  # Use ReLU activation
net.add(FCLayer(100, 50))
net.add(ActivationLayer(relu, relu_prime))  # Use ReLU activation
net.add(FCLayer(50, 10))
net.add(ActivationLayer(relu, relu_prime))
net.add(DropoutLayer(p=0.1))  # Add dropout layer
net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=20, learning_rate=0.1)
out = net.predict(x_test[0:10])
print("\n")
print("predicted values : ")
print(out, end="\n")
print("true values : ")
print(y_test[0:10])